### Dataset untuk training dihasilkan dengan menggunakan GPT-4 dari API OpenAI
Saya menggunakan API GPT-4 dari OpenAI untuk membuat dataset dengan informasi yang lebih akurat tanpa adanya halusinasi AI. Penggunaan GPT-4 memungkinkan saya untuk menghasilkan data yang lebih terpercaya dan dapat diandalkan dalam skala besar.

### Estimasi Biaya untuk 100 Ribuan Baris Data
Dengan menggunakan GPT-4 untuk menghasilkan dataset sebanyak 100 ribu baris data, estimasi biayanya berkisar antara $10 hingga $25. Biaya tersebut bergantung pada model spesifik yang digunakan, frekuensi penggunaan, dan perincian tarif dari API OpenAI.

### Alasan Tidak Melakukan Web Scraping
Saya memilih untuk tidak melakukan web scraping untuk mendapatkan dataset karena ingin menghindari pelanggaran terhadap Term of Service (ToS) atau aturan penggunaan dari situs web yang di-scrape. Melanggar ToS bisa berakibat pada sanksi hukum, oleh karena itu saya memilih untuk menggunakan API GPT-4 dari OpenAI sebagai alternatif yang sah dan legal untuk menghasilkan dataset yang dibutuhkan.

In [ ]:
import openai
import winsound

openai.api_key = 'sk-pzEuO5Dv7Yxxxxxxxxxxxxxxxxxxxxxxxxxx'

def play_error_sound():
    try:
        winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
    except Exception as e:
        print(f"Error playing sound: {e}")


for i in range(0, 1000):
    print(i + "Request berhasil")

    try:
        respons = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": f"""Buat pertanyaan seputar kesehatan sebanyak 100 pertanyaan beserta jawaban. berikan informasi yang valid dan fakta. selalu sarankan untuk pergi kedokter jika keadaan atau keluhan serius.
                 output yang di harapkan adalah sebagai berikut :
                 pertanyaan :
                 jawaban :
                 pertanyaan :
                 jawaban :
                 """}
            ])

        content_only = respons['choices'][0]['message']['content']
        with open('output.txt', 'a', encoding='utf-8') as output_file:
            output_file.write(content_only + "\n")
    except Exception as e:
        print(f"Error: {e}")
        play_error_sound()

## Lakukan data cleaning, dengan menghapus output yang tidak di harapkan, lakukan pengecekan secara manual, dikarenakan data yang dihasilkan pasti akan berbeda beda.

In [ ]:
input_file_path = 'output.txt'
output_file_path = 'dataset.txt'

with open(input_file_path, 'r') as input_file:
    lines = input_file.readlines()

filtered_lines = []

for i in range(len(lines) - 1):
    if lines[i].strip() == 'pertanyaan :' and lines[i + 1].strip() == 'jawaban :':
        filtered_lines.extend([lines[i], lines[i + 1]])

with open(output_file_path, 'w') as output_file:
    output_file.writelines(filtered_lines)

## Ambil sampel sekitar 10% dari data untuk di evaluasi fakta dan kebenarannya secara manual.

In [ ]:
import random
file_path = 'dataset.txt'

with open(file_path, 'r') as file:
    all_lines = file.readlines()

total_lines = len(all_lines)
num_random_lines = 2000

if total_lines <= num_random_lines:
    selected_lines = all_lines
else:
    random_indexes = random.sample(range(total_lines), num_random_lines)
    selected_lines = [all_lines[i] for i in random_indexes]

for line in selected_lines:
    print(line.strip())

## Train test split

In [ ]:
from sklearn.model_selection import train_test_split

with open("dataset.txt", "r") as file:
    data = file.readlines()

test_size = int(len(data) * 0.1)
test_start_idx = (len(data) - test_size) // 2
test_end_idx = test_start_idx + test_size

train = data[:test_start_idx] + data[test_end_idx:]
test = data[test_start_idx:test_end_idx]

with open("train_dataset.txt", "w") as file:
    file.writelines(train)

with open("test_dataset.txt", "w") as file:
    file.writelines(test)

print("Training set size:", len(train))
print("Testing set size:", len(test))

## Inisialisasi tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indonesian-nlp/gpt2-medium-indonesian")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

## load dataset

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=64)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=64)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

## Inisiasi Parameter Training

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("indonesian-nlp/gpt2-medium-indonesian")


training_args = TrainingArguments(
    output_dir="./finetune",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500,
    prediction_loss_only=True,
    logging_dir="./logs",
    logging_steps=100, 
    logging_first_step=False,
    save_total_limit=5,
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

## Training dan save model
Training model membutuhkan GPU dengan vram minimal 16gb.

In [6]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.948100
1000,2.704900
1500,2.628100
2000,2.583200
2500,2.548900
3000,2.525200
3500,2.469500
4000,2.406400
4500,2.400300
5000,2.387600


TrainOutput(global_step=9813, training_loss=2.4421457997960294, metrics={'train_runtime': 15987.9917, 'train_samples_per_second': 19.637, 'train_steps_per_second': 0.614, 'total_flos': 3.64456978808832e+16, 'train_loss': 2.4421457997960294, 'epoch': 3.0})

In [7]:
trainer.save_model()